In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.ml import Pipeline
from pipline.CastColumnsTransformer import CastColumnsTransformer
from pyspark.sql.functions import to_timestamp, monotonically_increasing_id, lit

In [2]:
# Démarrer une session Spark
spark = SparkSession.builder.appName("SupplyChain").getOrCreate()

# Lire le CSV depuis HDFS ou local
dfs = spark.read.option("header", "true").csv("hdfs://localhost:9000/projet/raw/supply_chain/DataCoSupplyChainDataset.csv")
dfs.show(5)


25/05/27 08:42:01 WARN Utils: Your hostname, toma-Nitro-AN517-52 resolves to a loopback address: 127.0.1.1; using 192.168.1.35 instead (on interface wlp0s20f3)
25/05/27 08:42:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/27 08:42:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/27 08:42:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+------------------------+-----------------------------+-----------------+------------------+----------------+------------------+-----------+--------------+-------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+------+----------------+----------------------+--------------+---------------+---------------+-------------+---------------+-------------------+-------------------+--------------------+------------+-------------+--------------+--------------------------+--------------+
|    Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order|Sales per c

In [3]:
train_df, test_df = dfs.randomSplit([0.7, 0.3], seed=42)

train_df.show(5)
test_df.show(5)

+----+------------------------+-----------------------------+-----------------+------------------+----------------+------------------+-----------+----------------+-------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------------+----------------+--------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+-----------+----------------+----------------------+---------------+----------------+------------+-------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------+--------------+--------------------------+-------------+
|Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order

+----+------------------------+-----------------------------+-----------------+------------------+----------------+------------------+-----------+----------------+-------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------+-------------+--------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+-----------+----------------+----------------------+---------------+----------------+------------+-------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------+--------------+--------------------------+-------------+
|Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order|Sales pe

In [4]:
cast_transformer = CastColumnsTransformer()

pipeline = Pipeline(stages=[cast_transformer])
model = pipeline.fit(dfs)

# ✅ Maintenant la sauvegarde doit marcher :
try:
    model.save("./big_data_pipeline_v3")
except Exception as e:
    model.write().overwrite().save("./big_data_pipeline_v3")

In [4]:
loaded_pipeline_model = PipelineModel.load("/home/toma/Documents/BIGDATA/big_data_pipeline_v3")
df = loaded_pipeline_model.transform(dfs)
lm_df = loaded_pipeline_model.transform(train_df)
df.show(10)  # Afficher les 5 premières lignes du DataFrame
df.describe().show()  # Afficher les statistiques descriptives du DataFrame
print(df.schema["order_date_(dateorders)"].dataType)  # Afficher le schéma du DataFrame


+--------+------------------------+-----------------------------+-----------------+------------------+-----------------+------------------+-----------+--------------+-------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+------+----------------+----------------------+--------------+---------------+---------------+-------------+---------------+-------------------+-------------------+--------------------+------------+-------------+--------------+--------------------------+--------------+
|    type|days_for_shipping_(real)|days_for_shipment_(scheduled)|benefit_per_order|sales_per_

+-------+--------+------------------------+-----------------------------+------------------+------------------+----------------+------------------+------------------+------------------+-------------+----------------+--------------+--------------+----------------+--------------+-----------------+----------------+------------------+---------------+-----------------+------------------+---------------+------------------+------------------+------+--------------+-------------+-----------------+------------------+----------------------+-------------------+------------------------+-----------------+------------------------+-----------------------+-------------------+------------------+------------------+----------------------+--------------+--------------+---------------+------------------+------------------+-------------------+-------------------+--------------------+--------------------+------------------+--------------+--------------+
|summary|    type|days_for_shipping_(real)|days_for_ship

In [5]:
train_df.write.mode("overwrite").csv(f"hdfs://localhost:9000/projet/gold/train_df", header=True)
test_df.write.mode("overwrite").csv(f"hdfs://localhost:9000/projet/gold/test_df", header=True)
train_df.show(5)
test_df.show(5)


+----+------------------------+-----------------------------+-----------------+------------------+----------------+------------------+-----------+----------------+-------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------------+----------------+--------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+-----------+----------------+----------------------+---------------+----------------+------------+-------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------+--------------+--------------------------+-------------+
|Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order

In [6]:
ft_trans = ["Type","Customer Id","Order Id","Product Card Id","Days for shipping (real)","Days for shipment (scheduled)","Benefit per order","Sales per customer","Delivery Status","Late_delivery_risk"]
ft_trans = [col.lower().replace(" ", "_") for col in ft_trans]
df_tans = lm_df.select(ft_trans)
df_tans = df_tans.dropDuplicates(["customer_id","order_id","product_card_id"])  # Supprimer le doublons
df_tans.show(10)  # Afficher les 5 premières lignes du DataFrame


+--------+-----------+--------+---------------+------------------------+-----------------------------+-----------------+------------------+-----------------+------------------+
|    type|customer_id|order_id|product_card_id|days_for_shipping_(real)|days_for_shipment_(scheduled)|benefit_per_order|sales_per_customer|  delivery_status|late_delivery_risk|
+--------+-----------+--------+---------------+------------------------+-----------------------------+-----------------+------------------+-----------------+------------------+
|   DEBIT|        100|   54995|            365|                       6|                            4|      46.24000168|       98.37999725|    Late delivery|                 1|
|   DEBIT|       1000|   47858|            365|                       4|                            2|      56.43999863|       117.5800018|    Late delivery|                 1|
|   DEBIT|      10000|   68882|            502|                       3|                            4|     -2.83999

In [7]:
ft_ord = ["Order Id","Customer Id","order_date_(dateorders)","shipping_date_(dateorders)","Order Status","Department Id","Shipping Mode","Order City","Order State","Order Country","Order Region","Market","Customer Zipcode"]
ft_ord = [col.lower().replace(" ", "_") for col in ft_ord]
df_ord = lm_df.select(ft_ord)
df_ord = df_ord.dropDuplicates(["order_id","customer_id"])
df_ord = df_ord.orderBy("shipping_date_(dateorders)")
df_ord.show(10)  # Afficher les 5 premières lignes du DataFrame
df_ord.describe().show()  # Afficher les statistiques descriptives du DataFrame

+--------+-----------+-----------------------+--------------------------+---------------+-------------+--------------+--------------------+-----------------+--------------------+---------------+------+----------------+
|order_id|customer_id|order_date_(dateorders)|shipping_date_(dateorders)|   order_status|department_id| shipping_mode|          order_city|      order_state|       order_country|   order_region|market|customer_zipcode|
+--------+-----------+-----------------------+--------------------------+---------------+-------------+--------------+--------------------+-----------------+--------------------+---------------+------+----------------+
|       1|      11599|    2015-01-01 00:00:00|       2015-01-03 00:00:00|         CLOSED|            7|Standard Class|         Mexico City| Distrito Federal|              M�xico|Central America| LATAM|           28601|
|      11|        918|    2015-01-01 03:30:00|       2015-01-03 03:30:00| PAYMENT_REVIEW|            4|Standard Class|      

+-------+-----------------+-----------------+---------------+------------------+--------------+--------------+--------------+-------------+--------------+------+------------------+
|summary|         order_id|      customer_id|   order_status|     department_id| shipping_mode|    order_city|   order_state|order_country|  order_region|market|  customer_zipcode|
+-------+-----------------+-----------------+---------------+------------------+--------------+--------------+--------------+-------------+--------------+------+------------------+
|  count|            58356|            58356|          58356|             58356|         58356|         58356|         58356|        58356|         58356| 58356|             58354|
|   mean|38294.32395983275| 7242.62144423881|           NULL| 5.495441771197478|          NULL|          NULL|          NULL|         NULL|          NULL|  NULL|35891.575247626555|
| stddev|22129.66133547846|4676.240963736433|           NULL|1.7866966550632668|          NULL|

In [8]:
ft_cust = ["Customer id","Customer fname","Customer lname","Customer email","Customer password","Customer segment","Customer street","Latitude","Longitude","Customer city","Customer state","Customer zipcode","Customer country"]
ft_cust = [col.lower().replace(" ", "_") for col in ft_cust]
df_cust = df.select(ft_cust)
df_cust = df_cust.dropDuplicates(["customer_id"])
df_cust = df_cust.orderBy("customer_id")
df_cust.describe().show()
df_cust.show(10)  # Afficher les 5 premières lignes du DataFrame

+-------+-----------------+--------------+--------------+--------------+-----------------+----------------+---------------+------------------+------------------+-------------+------------------+-----------------+----------------+
|summary|      customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_segment|customer_street|          latitude|         longitude|customer_city|    customer_state| customer_zipcode|customer_country|
+-------+-----------------+--------------+--------------+--------------+-----------------+----------------+---------------+------------------+------------------+-------------+------------------+-----------------+----------------+
|  count|            20652|         20652|         20644|         20652|            20652|           20652|          20652|             20652|             20652|        20652|             20652|            20649|           20652|
|   mean|10400.28592872361|          NULL|          NULL|          NULL|        

+-----------+--------------+--------------+--------------+-----------------+----------------+--------------------+-----------+------------+-------------+--------------+----------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_segment|     customer_street|   latitude|   longitude|customer_city|customer_state|customer_zipcode|customer_country|
+-----------+--------------+--------------+--------------+-----------------+----------------+--------------------+-----------+------------+-------------+--------------+----------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|        Consumer|  6303 Heather Plaza|25.95364761| -97.5076828|  Brownsville|            TX|           78521|         EE. UU.|
|         10|       Melissa|         Smith|     XXXXXXXXX|        XXXXXXXXX|       Corporate|8598 Harvest Beac...|38.47135925|-77.41419983|     Stafford|            VA|           22554|   

In [9]:
ft_cat = ["Product Category Id","Category Name"]
ft_cat = [col.lower().replace(" ", "_") for col in ft_cat]
df_cat = df.select(ft_cat)
df_cat = df_cat.dropDuplicates(["product_category_id"])
df_cat = df_cat.orderBy("product_category_id")
df_cat.show(10)  # Afficher les 5 premières lignes du DataFrame

+-------------------+-------------------+
|product_category_id|      category_name|
+-------------------+-------------------+
|                 10|  Strength Training|
|                 11|Fitness Accessories|
|                 12|       Boxing & MMA|
|                 13|        Electronics|
|                 16|    As Seen on  TV!|
|                 17|             Cleats|
|                 18|     Men's Footwear|
|                  2|             Soccer|
|                 24|    Women's Apparel|
|                 26|     Girls' Apparel|
+-------------------+-------------------+
only showing top 10 rows



In [10]:
ft_prod = ["Product Card Id","Product Category Id","Department Id","Product Name","Product Description","Product Price","Product Image","Product Status"]
ft_prod = [col.lower().replace(" ", "_") for col in ft_prod]
df_prod = df.select(ft_prod)
df_prod = df_prod.dropDuplicates(["product_card_id"])
df_prod = df_prod.orderBy("product_card_id")
df_prod.show(10)  # Afficher les 5 premières lignes du DataFrame

+---------------+-------------------+-------------+--------------------+-------------------+-------------+--------------------+--------------+
|product_card_id|product_category_id|department_id|        product_name|product_description|product_price|       product_image|product_status|
+---------------+-------------------+-------------+--------------------+-------------------+-------------+--------------------+--------------+
|             19|                  2|            2|Nike Men's Finger...|               NULL|  124.9899979|http://images.acm...|             0|
|             24|                  2|            2|Elevation Trainin...|               NULL|  79.98999786|http://images.acm...|             0|
|             35|                  3|            2|adidas Brazuca 20...|               NULL|  159.9900055|http://images.acm...|             0|
|             37|                  3|            2|adidas Kids' F5 M...|               NULL|  34.99000168|http://images.acm...|             0|

In [11]:
ft_depart = ["Department Id","Department Name"]
ft_depart = [col.lower().replace(" ", "_") for col in ft_depart]
df_depart = df.select(ft_depart)
df_depart = df_depart.dropDuplicates(["department_id"])
df_depart = df_depart.orderBy("department_id")
df_depart.show()  # Afficher les 5 premières lignes du DataFrame
df_depart.describe().show()  # Afficher les statistiques du DataFrame

+-------------+------------------+
|department_id|   department_name|
+-------------+------------------+
|            2|           Fitness|
|            3|          Footwear|
|            4|           Apparel|
|            5|              Golf|
|            6|          Outdoors|
|            7|          Fan Shop|
|            8|         Book Shop|
|            9|        Discs Shop|
|           10|        Technology|
|           11|          Pet Shop|
|           12|Health and Beauty |
+-------------+------------------+

+-------+---------------+---------------+
|summary|  department_id|department_name|
+-------+---------------+---------------+
|  count|             11|             11|
|   mean|            7.0|           NULL|
| stddev|3.3166247903554|           NULL|
|    min|              2|        Apparel|
|    max|             12|     Technology|
+-------+---------------+---------------+



In [12]:
ft_ord_item = ["Order Item Id","Order Id","product Card Id","Order Item Quantity","Order Item Discount","Order Item Discount Rate","Order Item Product Price","Order Item Profit Ratio","Order Item Total"]
ft_ord_item = [col.lower().replace(" ", "_") for col in ft_ord_item]
df_ord_item = lm_df.select(ft_ord_item)
df_ord_item = df_ord_item.dropDuplicates(["order_item_id","order_id","product_card_id"])
df_ord_item = df_ord_item.orderBy("order_id")
df_ord_item.show(10)  # Afficher les 5 premières lignes du DataFrame
df_ord_item.describe().show()  # Afficher les statistiques du DataFrame

+-------------+--------+---------------+-------------------+-------------------+------------------------+------------------------+-----------------------+----------------+
|order_item_id|order_id|product_card_id|order_item_quantity|order_item_discount|order_item_discount_rate|order_item_product_price|order_item_profit_ratio|order_item_total|
+-------------+--------+---------------+-------------------+-------------------+------------------------+------------------------+-----------------------+----------------+
|            1|       1|            957|                  1|               60.0|             0.200000003|              299.980011|            0.370000005|     239.9799957|
|           24|      10|           1073|                  1|               50.0|                    0.25|             199.9900055|            0.479999989|     149.9900055|
|           28|      10|           1073|                  1|               40.0|             0.200000003|             199.9900055|          

+-------+------------------+------------------+------------------+-------------------+-------------------+------------------------+------------------------+-----------------------+------------------+
|summary|     order_item_id|          order_id|   product_card_id|order_item_quantity|order_item_discount|order_item_discount_rate|order_item_product_price|order_item_profit_ratio|  order_item_total|
+-------+------------------+------------------+------------------+-------------------+-------------------+------------------------+------------------------+-----------------------+------------------+
|  count|            126634|            126634|            126634|             126634|             126634|                  126634|                  126634|                 126634|            126634|
|   mean| 90257.39272233365| 36220.51837579165| 692.1132871108865| 2.1293728382582877|  20.62062504480788|     0.10154358278888787|      141.18766088377143|    0.11964298682508574|183.15224014340697|


In [13]:
ft_deliv = ["Order Id","Type","Days for shipping (real)","Days for shipment (scheduled)","Delivery Status","Late_delivery_risk","Benefit per order","Order Profit Per Order"]
ft_deliv = [col.lower().replace(" ", "_") for col in ft_deliv]
df_deliv = lm_df.select(ft_deliv)
df_deliv = df_deliv.dropDuplicates(["order_id"])
df_deliv = df_deliv.orderBy("order_id")
df_deliv.show(10)  # Afficher les 5 premières lignes du DataFrame
df_deliv.describe().show()  # Afficher les statistiques du DataFrame

+--------+--------+------------------------+-----------------------------+----------------+------------------+-----------------+----------------------+
|order_id|    type|days_for_shipping_(real)|days_for_shipment_(scheduled)| delivery_status|late_delivery_risk|benefit_per_order|order_profit_per_order|
+--------+--------+------------------------+-----------------------------+----------------+------------------+-----------------+----------------------+
|       1|    CASH|                       2|                            4|Advance shipping|                 0|      88.79000092|           88.79000092|
|      10| PAYMENT|                       6|                            4|   Late delivery|                 1|      28.72999954|           28.72999954|
|     100|TRANSFER|                       6|                            2|   Late delivery|                 1|           -13.25|                -13.25|
|    1000|    CASH|                       6|                            2|   Late delive

+-------+------------------+--------+------------------------+-----------------------------+----------------+-------------------+------------------+----------------------+
|summary|          order_id|    type|days_for_shipping_(real)|days_for_shipment_(scheduled)| delivery_status| late_delivery_risk| benefit_per_order|order_profit_per_order|
+-------+------------------+--------+------------------------+-----------------------------+----------------+-------------------+------------------+----------------------+
|  count|             58356|   58356|                   58356|                        58356|           58356|              58356|             58356|                 58356|
|   mean| 38294.32395983275|    NULL|       3.500445541161149|            2.933751456576873|            NULL|  0.548221262595106| 12.19693435128885|     12.19693435128885|
| stddev|22129.661335478577|    NULL|      1.6244057377352135|           1.3754390058316655|            NULL|0.49767354170124173|120.0012812

In [14]:
def write_to_hdfs(df, path):
    """
    Écrit le DataFrame dans un fichier CSV sur HDFS.
    """
    df.write.mode("overwrite").csv(f"hdfs://localhost:9000/projet/{path}", header=True)

In [15]:
write_to_hdfs(df_cust, "gold/Customer")
write_to_hdfs(df_prod, "gold/Product")
write_to_hdfs(df_depart, "gold/Department")
write_to_hdfs(df_cat, "gold/Category")

In [16]:

write_to_hdfs(df_tans, "gold/Transaction")
write_to_hdfs(df_deliv, "gold/Delivery")
write_to_hdfs(df_ord, "gold/Order")
write_to_hdfs(df_ord_item, "gold/OrderItem")

In [17]:
spark.stop()